In [ ]:
import sys, os, copy, datetime
sys.path.append('/content')
from src.fluvius import WaterData, WaterStation
import pandas as pd

env_vars = open("/content/credentials","r").read().split('\n')

for var in env_vars[:-1]:
    key, value = var.split(' = ')
    os.environ[key] = value
    
storage_options= {'account_name':os.environ['ACCOUNT_NAME'],
                    'account_key':os.environ['BLOB_KEY']}

In [ ]:
# First, generate WaterStation data but add a row for each day of the year

# Initialize WaterData to house new stations
db = WaterData("itv", "itv-data", 500, storage_options)
db.get_source_df()

In [ ]:
db.get_station_data()
db.df

In [ ]:
focal_station = db.df["site_no"][2]

In [ ]:
my_station = db.station[focal_station]
my_station.df

In [ ]:
start_date = datetime.date.fromisoformat("2015-09-01")
end_date = datetime.date.fromisoformat("2021-12-15")

daterange = pd.date_range(
    start_date,
    end_date - datetime.timedelta(days=1),
    freq='d').to_pydatetime()

dates = [x.date() for x in daterange]

lat = [my_station.df["Latitude"][0]] * len(dates)
lon = [my_station.df["Longitude"][0]] * len(dates)

sample_ids = [f"{my_station.site_no.zfill(8)}_pred_{str(i).zfill(8)}" for i in range(0, len(dates))]

my_station.df = pd.DataFrame({
    "sample_id": sample_ids,
    "Longitude": lon,
    "Latitude": lat,
    "Date-Time": dates
})

my_station.df
my_station.time_of_interest = f"{start_date}/{end_date}"
my_station.build_catalog()
my_station.get_cloud_filtered_image_df(80)

In [ ]:
my_station.merge_image_df_with_samples(0)

In [ ]:
my_station.perform_chip_cloud_analysis()

In [ ]:
my_station.get_chip_features(False, "", "lulc", "mndwi")

In [ ]:
my_station.merged_df.to_csv("/content/local/predictions/itv3_features.csv")